# SSC Data Science and Analytics Workshop 2021

## The Data Scientist’s Workflow: EDA and Statistical Modeling with Python in Jupyter Notebooks

### Nathan Taback

# Topics

In the next 40 minutes we will to cover ...

- Data wrangling: 
   - selecting variables
   - filtering rows
   - creating new variables
   - grouping
   - combining multiple data frames

- Summarizing Data 
- Visualizing Data

# Data Wrangling

- Data wrangling/manipulation/transformation has a large impact on the data used to answer questions with data.

- Statistical and machine learning models are meaningful only if the data is meaningful.

- Data wrangling/manipulation is one point in the data analysis process where decisions can introduce bias into the data.  Examples?

<a href='https://www.nytimes.com/interactive/2019/01/11/us/politics/trump-border-crisis-reality.html'> <img src='trumpnyt.png'> </img> </a>

![](pandas_screenshot.png)

# Series

A Series is a one-dimensional array-like object containing a sequence of values (of similar types to NumPy types) and an associated array of data labels, called its index. The simplest Series is formed from only an array of data

In [ ]:
import pandas as pd

myseries = pd.Series([0.9, 0.7, -10, 20])
print(myseries)
prov = ['Manitoba', 'Ontario', 'Quebec', 'Alberta']
myseries.index = prov
myseries

- `myseries` is now indexed by `prov`.
- the value for Alberta can be obtained by `myseries['Alberta']`

<h1> Your turn  &#128073; </h1> 



1. Create a new cell.

2. Access the value for Ontario.

# pandas DataFrame

- A DataFrame represents a rectangular table of data and contains an ordered collection of columns, each of which can be a different value type (numeric, string, boolean, etc.). 

- The DataFrame has both a row and column index; it can be thought of as a dict of Series all sharing the same index.

- Under the hood, the data is stored as one or more two-dimensional blocks rather than a list, dict, or some other collection of one-dimensional arrays. 

(McKinney, 2018 and [pandas ref](https://pandas.pydata.org))

- a pandas cheat sheet is available [here](https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf).

In [ ]:
# a dict of equal lengths

pop = {'province':['Ontario', 'Ontario', 'Quebec', 'Quebec'], 
        'year': [2016, 2011,2016, 2011],
        'pop_size': [13448494, 12851821, 8164361, 7903001]}
df = pd.DataFrame(pop)
df

<h1> Your turn  &#128073; </h1> 



1. Create a new cell, 

2. add this data to `df`,

province | population | year
---------| -----------| -----
Manitoba |  1,278,365 |  2016
Manitoba |  1,208,268 | 2011

3. Display the data frame

- pandas can read and write data in many types of formats (text, binary, SQL) that are stored as csv, json, html, etc.  (see [IO tools](http://pandas.pydata.org/pandas-docs/stable/io.html) )

- For example, `read_html` accepts an HTML string/file/URL and will parse an HTML tables into a list of pandas DataFrames.

# Population of Countries

![text](wikipedia_poppage.png)

In [ ]:
url = 'https://en.wikipedia.org/wiki/List_of_countries_by_population_(United_Nations)'
df = pd.read_html(url, header=0)
type(df)

In this case `pd.read_html()` returns a list of data frames.  By inspection, the first element of the list contains the Wikipedia table we want.

In [ ]:
pop = df[0] # select table of countries
pop

The data is sorted by 2019 world population so we can use index to create a rank variable.

In [ ]:
pop.reset_index(inplace = True) #create a column from index values
pop = pop.rename(columns = {'index': 'rank'}) # rename the column
pop.head() # display the first 6 rows

<h1> Your turn  &#128073; </h1> 

1. Create a new cell.

2. Read the tables from <https://en.wikipedia.org/wiki/List_of_Arab_countries_by_population> into a pandas data frame.

3. Display the data frame with the Past and Future table. 

![](arabcount_screenshot.png)

## Dataframe properties

- data frame dimensions: `pop.shape`

- column names: `list(pop)` or `pop.columns`

In [ ]:
print('Shape of the data frame is:', pop.shape) 
print('\n')
print('Column names:',list(pop))

<p> &#9995; Indexing starts from 0 (unlike R &#128560;) so we should add 1 to <code>rank</code>.</p> 

In [ ]:
pop['rank'] = pop['rank'].apply(lambda x: x + 1) # apply with an anonymous fucntion 
pop.head()

# Filtering rows (observations)

Select countries with an *index* value between 230 and 234 inclusive

In [ ]:
pop[230:234]

- Let's remove the last row since it's not a country. 
- Use `DataFrame.drop` and modify the data frame with `inplace = True`.

In [ ]:
pop.drop(233, inplace = True)

In [ ]:
print('There are', pop.shape[0], 'rows and' ), 
pop[232:]

# `.loc` and `.iloc`

- Use `.loc` and `.iloc` to select by rows and columns.

- `.loc` is primarily label based.

- `.iloc` is primarily integer based.

- What is the change in population for the country of rank 30?

In [ ]:
print('using loc:', pop.loc[29,'Change'], '\n')
print('using iloc:', pop.iloc[29, 6])

- Modify values in place using `.loc` 

In [ ]:
pop.loc[29,'Change'] = 0.09
print(pop.loc[29,'Change']),
type(pop.loc[29,'Change'])

- `iloc` is primarily integer based
- We can also select rows using integers with `iloc`.  For example, the 7th column has index value 7 - 1 = 6 (i.e., from 0 to length of axis - 1).

In [ ]:
pop.iloc[29,6]

# Selecting variables (columns)


- `pop['rank']` returns a pandas series (i.e., data frame with one column).


In [ ]:
pop['rank'].head()

**Question:** Which Asian countries are in the top 10 by rank?  What is the distribution of population change in these countries?

In [ ]:
print(pop.columns) 
pop[pop.columns[2]].head()

In [ ]:
t1 = pop.loc[0:2,'Country/Territory':'UN continentalregion[4]']
t2 = pop.iloc[0:3, 0:3]
t3 = pop[['Country/Territory','UN continentalregion[4]']][0:3]
display(t1, t2, t3)


We can also use boolean logic to select rows.

In [ ]:
pop[pop.columns[2] == 'Asia' & pop['rank'] <= 10]

In [ ]:
pop.dtypes

object dtype, can hold any Python object, including strings (see [pandas ref](https://pandas.pydata.org/pandas-docs/stable/user_guide/basics.html#defaults)). But, we can to convert variable to a string type using `astype`.

In [ ]:
pop[pop.columns[2]].astype('string').head()

In [ ]:
pop[(pop[pop.columns[2]].astype('string') == 'Asia') & (pop['rank'] <= 10)]

`Change` is type object so we will need to: 

1. convert to a string, 
2. extract the digits, 
3. convert digits to type numeric, then  
4. compute distribution.



In [ ]:
# notice that the entire expression below is enclosed in () 
# improves readability

(pop[(pop[pop.columns[2]] == 'Asia') & (pop['rank'] <= 10)]['Change'].
astype('string'). # 1. convert to string
str.extract(r'(\d\.\d+|-\d\.\d+)'). # 2. extract decimal digits
astype('float'). # 3. convert to numeric
describe()) # 4. compute distribution

- This is an example where we call methods on an object one after another - sometimes called [method chaining](https://tomaugspurger.github.io/method-chaining).  

- Similar in style to `%>%` in R `tidyverse`.

# Creating New Variables

Suppose we want to add a new variable to the data frame to indicate if a countries' change in population has magnitude greater than 3% (i.e., change $\geq$ 3% or change $\leq$ -3%).


In [ ]:
pop['Change'].head(12)

Use `-\d+\.\d+|\d+\.\d+` to extract positive and negative digits.

In [ ]:
pop['Change'].astype('string').str.extract(r'(-\d+\.\d+|\d+\.\d+)').head(12)

- Why didn't the regex work for the minus sign?
- Look at unicode character for index 10.

In [ ]:
print(ord(pop['Change'].astype('string')[10][0]))
ord('-') # minus sign

- Minus sign has integer 45.
- So, need to replace unicode 8722 with unicode 45.

In [ ]:
ch1 = ord(pop['Change'].astype('string')[10][0])

pop['Change'] = (pop['Change'].astype('string').
                 str.replace(chr(ch1),'-').
                 str.extract(r'(-\d+\.\d+|\d+\.\d+)')[0].
                 astype(float))

pop['Change']


- Now, need to apply a transformation to select changes between -3 and 3.
- One way to do it is to use `apply` with a `lambda` function.

In [ ]:
pop['Change3'] = pop['Change'].apply(lambda x: 1 if (x >= -3.0 and x <= 3.0) else 0)

pop['Change3']

In [ ]:
pop.groupby(['Change3']).count()

# Statistics Canada Daily Indicators

## How is Canada's Economy today?

![text](statcan1.png)

In [ ]:
dailydat = 'https://www150.statcan.gc.ca/n1/dai-quo/ssi/homepage/ind-econ.json'

df = pd.read_json(dailydat)
df.head()

In [ ]:
type(df['results']['indicators'])

In [ ]:
df['results']['indicators'][0]

Two ways to flatten the json file.

1. use pandas `json_normalize`

2. iterate through list

In [ ]:
indicators = pd.json_normalize(df['results']['indicators'])

In [ ]:
indicators.head()

- Some of the indicators are not reported for all provinces/territories.
- Create a list of indicators in the daily that have at `geo_code` $\ge 1$.

In [ ]:
l = list(indicators.groupby(by=['title.en', 'geo_code']).groups)

- Use a list comprehension to create a list of indicators that have at least two geo_codes

In [ ]:
titles_weekly = [title for title, geo in l if geo >= 1] # list of titles for each indicator

- remove duplicate titles

In [ ]:
res = [] 

[res.append(t) for t in titles_weekly if t not in res] # only keep one copy

res

We want to plot the indicator on a map of Canada s let's create a Chloropleth map.  For this type of map we will need a file with geometry of Canadian provinces and Territories as polygons (i.e., a geojson file).

![](canadageojson.png)

- use names from geojson to create dictionary with `geo_code` from `weeklyearnings` (i.e., used in stats can data).
- merge `weekly_earnings` with `canada_geo`.
- use `geopandas` to import geojson file of Canada then extract names of provinces.
- we will need to merge these names with `geo_code` from `weeklyearnings`.

In [ ]:
import geopandas

geourl = 'https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/canada.geojson'

geocanada = geopandas.read_file(geourl)

geocanada.head()

- Create a dictionary where **key** is provience/territory name from geojson, and **value** is `geo_code` value from Statistics Canada data.

In [ ]:
geo_dict = {'Canada' : [0], 
            geocanada['name'][1] : [1],
            geocanada['name'][9]: [2],
            geocanada['name'][6] : [3],
            geocanada['name'][5] : [4],
            geocanada['name'][0] : [5],
            geocanada['name'][12] : [6],
            geocanada['name'][11] : [7],
            geocanada['name'][7] : [8],
            geocanada['name'][8] : [9],
            geocanada['name'][2] : [10],
            geocanada['name'][10] : [11],
            geocanada['name'][4] : [12],
            geocanada['name'][3] : [13]}

# index is set as key
canada_geo = pd.DataFrame.from_dict(geo_dict, orient = 'index', columns = ['geo_code'])

# reset index to create a column for province/territory name
canada_geo.reset_index(inplace=True)

# rename column
canada_geo = canada_geo.rename(columns = {'index':'region'})
canada_geo

# Data Visualization

What are the relationships between growth of economic indicators?

Consider the relationship between growth rate of `Building permits` and `Average weekly earnings`.

In [ ]:
# growth rate for building permits
indicators[indicators['title.en'] == res[1]][['geo_code','growth_rate.growth.en']].head()

In [ ]:
indicators['growth_rate.growth.en'].dtypes

In order to plot `growth_rate.growth.en` need to convert to numeric type. One way to do this is:

1. convert to a string `astype('string')`
2. replace `%` with '' `str.replace('%','')`
3. convert to numeric `pd.to_numeric()`

In [ ]:
v1 = indicators[indicators['title.en'] == res[1]][['geo_code','growth_rate.growth.en']]
v1['growth_rate.growth.en'] = pd.to_numeric(v1['growth_rate.growth.en'].astype('string').str.replace('%',''))

v2 = indicators[indicators['title.en'] == res[0]][['geo_code','growth_rate.growth.en']]
v2['growth_rate.growth.en'] = pd.to_numeric(v2['growth_rate.growth.en'].astype('string').str.replace('%',''))

- Merge `v1` and `v2` by `geo_code` then
- Merge the result with `canada_geo` data frame so that we can label points by province names.

In [ ]:
df = v1.merge(v2, on = 'geo_code', suffixes = (res[0],res[1])).merge(canada_geo, on = 'geo_code')

Plot the variables using pyplot API from matplotlib `matplotlib.pyplot`.

In [ ]:
import matplotlib.pyplot as plt

x1 = df[list(df)[2]]
y1 = df[list(df)[1]]

# grab first four letters of provinces
labs = df['region'].astype('string').str.slice(start=0,stop=4)


plt.scatter(x = x1 , y = y1)
plt.xlabel(list(df)[2])
plt.ylabel(list(df)[1])

# add annotation
for i, txt in enumerate(labs):
    plt.annotate(txt, (x1[i], y1[i]), ha = 'left')

Same plot using matplotlib object-oriented API - this API allows finer control over plots.

In [ ]:
fig, ax = plt.subplots()
ax.plot(x1, y1, 'ro') # red circles
ax.set_xlabel(list(df)[2])
ax.set_ylabel(list(df)[1])
for i, txt in enumerate(labs):
    ax.annotate(txt, (x1[i], y1[i]), ha = 'left')
plt.show()

We could also plot directly using pandas.

In [ ]:
df.plot.scatter(x = list(df)[2],y = list(df)[1])

Define a function to extract growth values of two indicators.

In [ ]:
# write a function that returns a data frame with growth values for two indicators 
def get_two_indicators(i1,i2):
    # create indicators v1, v2
    v1 = indicators[indicators['title.en'] == i1][['geo_code','growth_rate.growth.en']]
    v1['growth_rate.growth.en'] = pd.to_numeric(v1['growth_rate.growth.en'].astype('string').str.replace('%',''))
    v2 = indicators[indicators['title.en'] == i2][['geo_code','growth_rate.growth.en']]
    v2['growth_rate.growth.en'] = pd.to_numeric(v2['growth_rate.growth.en'].astype('string').str.replace('%',''))
    # return merged data frame
    return v1.merge(v2, on = 'geo_code', suffixes = (i2,i1)).merge(canada_geo, on = 'geo_code')
    
get_two_indicators(res[0],res[1])

Create a dataframe with four indicators using `get_two_indicators` and `DataFrame.merge`.

In [ ]:
ind_basket = ((get_two_indicators(res[0],res[1]).
              merge(get_two_indicators(res[2],res[3]), on = 'geo_code')).
              iloc[:,[1,2,4,5]])
ind_basket.head()

- Let's clean up column names, say by, using the last word.  
- The regex `\s(\w+)$` will extract the last word of the column names.

In [ ]:
import re

#put column names in a list
long_names = list(ind_basket)

# search for last word in title and put in a list
short_names = [re.search('\s(\w+)$',col_name).group() for col_name in long_names]

short_names

- Now rename columns in data frame with shorter versions.
- Use `pandas.DataFrame.rename`.  
- Need a dictionary of (key:value) pairs, where key = long_names and value = short_names
- `zip` function is an iterator of tuples where first item in each iterator is paired with second item.

In [ ]:
short_names_dict = dict(zip(list(ind_basket),short_names))
short_names_dict

Now rename columns with `inplace=True`.

In [ ]:
ind_basket.rename(columns = short_names_dict, inplace=True)

In [ ]:
pd.plotting.scatter_matrix(ind_basket)
plt.show()

# Interactive Plots Using jupyter-widgets

Let's modify our function to return a scatter plot instead of a data frame. 

In [ ]:
def plot_two_indicators(i1,i2):
    # create plotting variables
    v1 = indicators[indicators['title.en'] == i1][['geo_code','growth_rate.growth.en']]
    v1['growth_rate.growth.en'] = pd.to_numeric(v1['growth_rate.growth.en'].astype('string').str.replace('%',''))
    v2 = indicators[indicators['title.en'] == i2][['geo_code','growth_rate.growth.en']]
    v2['growth_rate.growth.en'] = pd.to_numeric(v2['growth_rate.growth.en'].astype('string').str.replace('%',''))
    # two merges into data frames 
    df = v1.merge(v2, on = 'geo_code', suffixes = (i2,i1)).merge(canada_geo, on = 'geo_code')
    cols = list(df)
    x1 = df[cols[2]]
    y1 = df[cols[1]]
    # first four letters of region for annotation 
    labs = df['region'].astype('string').str.slice(start=0,stop=4)
    fig1 = plt.subplots()
    plt.scatter(x=x1, y = y1)
    plt.xlabel(cols[2])
    plt.ylabel(cols[1])
    # annotate each point
    for i, txt in enumerate(labs):
        plt.annotate(txt, (x1[i], y1[i]), ha = 'right')
    plt.show(fig1)


plot_two_indicators(res[4],res[5])

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact

display(widgets.HTML(value="<h1>Select the indicators to plot</h1>"))

interact(plot_two_indicators, 
         i1 = widgets.Dropdown(options = res, value = res[0], decription = "Indicator 1"),
         i2 = widgets.Dropdown(options = res, value = res[1], decription = "Indicator 2"));

# Plot Weekly Earnings on a Map of Canada

Let's pick 'Average weekly earnings' or `res[0]`.

Create data frame:

- group by indicator `DataFrame.groupby`
- extract groups using `get_groups`

In [ ]:
groups = indicators.groupby(by=['title.en'])
weeklyearnings = groups.get_group(res[0])

In [ ]:
prov_weeklyearnings = weeklyearnings.merge(canada_geo, how = 'left', on = 'geo_code')
prov_weeklyearnings['value.en']

Replace `,` and `$` with `''` and convert to numeric

In [ ]:
prov_weeklyearnings['value.en'] = (pd.to_numeric(prov_weeklyearnings['value.en'].
                                                 astype('string').
                                                 str.replace(r'(,|\$)','')))

In [ ]:
# select two columns for plotting on map
prov_earnings = prov_weeklyearnings[['value.en','region']]

# drop value for Canada
prov_earnings.drop(12, axis = 0, inplace=True)

- Create a map centred on North America using `folium` library.
- create a map centred on North America

In [ ]:
import folium
m = folium.Map(location=[48, -102], zoom_start=3)
m

Now create Choropleth layer that adds weekly earnings.

In [ ]:
folium.Choropleth(
    geo_data=geourl, #geourl
    name="choropleth",
    data = prov_earnings,
    columns = ['region','value.en'],
    key_on = "feature.properties.name",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Weekly Earnings").add_to(m)

folium.LayerControl().add_to(m)

m